In [1]:
!cd "/content/drive/MyDrive/BDMA/Semester4 Thesis/Ainslie-surrogate"

zsh:cd:1: no such file or directory: /content/drive/MyDrive/BDMA/Semester4 Thesis/Ainslie-surrogate


In [2]:
import gpytorch
import torch

from utils.data_loaders import get_wake_datasets

In [3]:
FACTORS_FOLDER = "discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01"
DATA_FOLDER = f"data/{FACTORS_FOLDER}/"
#MODEL_NAME = f"univariate_GP" #TODO make the name represent the structure or the parameters of the model
#BEST_MODEL_PATH = f"saved_models/{FACTORS_FOLDER}/{MODEL_NAME}.pt"
CONSIDER_WS = False
COORDS_AS_INPUT = True # univariate setting

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{DEVICE=}")

# hyperparameters
EPOCHS = 2 #TODO
LR = 0.0001

DEVICE=device(type='cpu')


In [4]:
valid_dataset = None
train_dataset, valid_dataset, test_dataset = get_wake_datasets(DATA_FOLDER,
                                                          consider_ws=CONSIDER_WS,
                                                          coords_as_input=COORDS_AS_INPUT,
                                                          train_perc=0.6,
                                                          test_perc=0.3,
                                                          validation_perc=0.1)

train_x, train_y = train_dataset.inputs, train_dataset.outputs.view(-1)
test_x, test_y = test_dataset.inputs, test_dataset.outputs.view(-1)
print("Train shapes: ", train_x.shape, train_y.shape)
print("Test shapes: ", test_x.shape, test_y.shape)

if valid_dataset is not None:
    valid_x, valid_y = valid_dataset.inputs, valid_dataset.outputs.view(-1)
    print("Valid shapes: ", valid_x.shape, valid_y.shape)

del test_x, test_y, test_dataset, valid_x, valid_y, valid_dataset

Train shapes:  torch.Size([36986880, 4]) torch.Size([36986880])
Test shapes:  torch.Size([18493440, 4]) torch.Size([18493440])
Valid shapes:  torch.Size([6164480, 4]) torch.Size([6164480])


In [5]:
from utils.data_utils import get_parameters_from
from utils.pywake_utils import get_grid_shape
x_start, x_end, y_start, y_end, grid_factor, _, _ = get_parameters_from(DATA_FOLDER)
grid_shape = get_grid_shape(x_start, x_end, y_start, y_end, grid_factor)
grid_size = grid_shape[0] * grid_shape[1]
print(grid_size)

num_instances = 2 #TODO
num_features = 4
train_x= torch.split(train_x, grid_size * num_instances)[0]#.reshape(num_instances, grid_size, num_features)
train_y = torch.split(train_y, grid_size * num_instances)[0]#.reshape(num_instances, grid_size)
print(train_x.shape, train_y.shape)

7168
torch.Size([14336, 4]) torch.Size([14336])


In [6]:
# the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([grid_size]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(batch_shape=torch.Size([grid_size])),
            batch_shape=torch.Size([grid_size])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood(batch_shape=torch.Size([grid_size]))
model = ExactGPModel(train_x, train_y, likelihood)
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [7]:
for i in range(EPOCHS):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    try:
        print(loss.shape)
    except:
        pass
    loss.backward()
    lengthscale = model.covar_module.base_kernel.lengthscale.item()
    noise = model.likelihood.noise.item()
    print(f"Iter {i+1}/{EPOCHS} - Loss: {loss.item:.3f}\tlengthscale: {lengthscale:.3f}\tnoise: {noise:.3f}")
    optimizer.step()

: 

: 